In [ ]:
! pip install openai tiktoken langchain gradio

In [1]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from elevenlabs import voices, generate, play, save
import constants
import openai

# Load API keys from environment variables
os.environ["OPENAI_API_KEY"] = constants.OPENAI_API_KEY
openai.api_key = constants.OPENAI_API_KEY

/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
def summarize_and_sonify(text, file, num_minutes, voice):
    
    if len(text) == 0:
        # Read doc
        doc = PyPDFLoader(file.name).load()

        # Create string from doc content
        text = ''
        for page in doc:
            text += page.page_content

    num_words = num_minutes*135 # Approximately 135 words per minute with TTS voice (refine this)

    audio_voice = voice.split()[0]
    summary_lang = "English"
    if voice.split()[1] == "(SWE)":
        summary_lang = "Swedish"
    if voice.split()[1] == "(IT)":
        summary_lang = "Italian"

    # Connect to OpenAI, prompt ChatGPT to summarize the text
    chat_model = ChatOpenAI() # Must have set API key as env var
    prompt = "Summarize the following text in {} words. The summary must be in {}. Text: {}".format(num_words, summary_lang, text) 
    summary = chat_model.predict(prompt)

    # Connect to elevenlabs, generate audio
    audio = generate(text=summary, voice=audio_voice, model="eleven_monolingual_v1", api_key=constants.ELEVEN_API_KEY)

    # Save audiofile
    if not os.path.exists("audiofiles"):
        os.makedirs("audiofiles")

    # Handling identical file names
    file_suffix = 0
    while os.path.exists("audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)):
        file_suffix +=1

    save(audio, "audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix))

    return "audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)




In [10]:
demo = gr.Interface(fn=summarize_and_sonify, 
                    inputs=[
                        "text",
                        "file", 
                        gr.Slider(minimum=0.5, maximum=5, step=0.5, label="Length", info = "Number of minutes of audio summary"),
                        gr.Dropdown(["Adam (US)", "Antoni (IT)", "Arnold (US)", "Bella (US)", "Domi (US)", "Elli (US)", "Josh (SWE)", "Rachel (US)", "Sam (US)"], label="Voice", info="Narrator voice and language of audio summary")
                            ],
                    outputs= ["audio"]
                    )
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://d28ff7cc79577478fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/gradio/routes.py", line 507, in predict
    output = await route_utils.call_process_api(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/gradio/route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1437, in process_api
    result = await self.call_function(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worke

In [4]:
voices = voices()
for i in range(30):
    print(voices[i].name)

Rachel
Clyde
Domi
Dave
Fin
Bella
Antoni
Thomas
Charlie
Emily
Elli
Callum
Patrick
Harry
Liam
Dorothy
Josh
Arnold
Charlotte
Matilda
Matthew
James
Joseph
Jeremy
Michael
Ethan
Gigi
Freya
Grace
Daniel


Traceback (most recent call last):
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/gradio/routes.py", line 507, in predict
    output = await route_utils.call_process_api(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/gradio/route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1437, in process_api
    result = await self.call_function(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/Users/danielbouvin/Library/Python/3.9/lib/python/site-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worke